In [1]:
import numpy as np
import torch
from tqdm import tqdm

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn
seaborn.set()

import reservoir
import reckernel
import kuramoto
%load_ext autoreload
%autoreload 2

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
today = '200609'

# MSE for time series prediction

We repeat it several times to obtain the average on several random realizations.

## Reservoir Computing

In [2]:
N_seed = 10

L = 22 / (2 * np.pi)  # length
N = 100  # space discretization step
dt = 0.25  # time discretization step
N_train = 70000
N_test = 5000
N_init = 1000  # remove the initial points
tend = (N_train + N_test) * dt + N_init

n_rec = 600
test_len = 2000
rc_mse = torch.zeros(test_len, n_rec+1, N_seed)

for seed in tqdm(range(N_seed)):
    np.random.seed(seed)
    dns = kuramoto.KS(L=L, N=N, dt=dt, tend=tend)
    dns.simulate()
    u = dns.uu[N_init:] / np.sqrt(N)
    [u_train, u_test, _] = np.split(u, [N_train, N_train+N_test], axis=0)
    u_train_t = torch.from_numpy(u_train).to(device)
    u_test_t = torch.from_numpy(u_test).to(device)
    input_len, input_dim = u_train_t.shape
    
    # RC parameters
    n_res = 1024-input_dim
    input_scale = 0.4
    res_scale = 0.9
    bias_scale = 0.4
    renorm_factor = 1.1 * np.sqrt(input_dim / n_res)
    alpha = 1e-2
    leak_rate = 1

    # Generation of train data
    pred_horizon_range = 1
    out_train = torch.zeros(input_len, input_dim * pred_horizon_range).to(device)
    for pred_horizon in range(1, pred_horizon_range+1):
        out_train[:, (pred_horizon-1)*input_dim:pred_horizon*input_dim] = torch.roll(u_train_t, -pred_horizon, dims=0)

#     print("Training forward")
    torch.manual_seed(seed)
    bias = bias_scale * torch.randn(n_res).to(device)
    model = reservoir.ESN(input_dim, res_size=n_res, res_scale=res_scale, input_scale=input_scale, 
                          f='erf', leak_rate=leak_rate, bias=bias, random_projection='gaussian', seed=seed)
    X = model.forward(u_train_t).to(device)
    X = torch.cat((X, u_train_t*renorm_factor), dim=1)
#     print("Training weights")
    output_w = model.train(X, out_train, alpha=alpha)
    
#     print("Testing forward")
    test_len = 2000
    Xtest = model.forward(u_test_t[:test_len]).to(device)

#     print("Testing recursive prediction")
    n_rec = 600
    rec_pred_data = model.rec_pred(Xtest, output_w, n_rec, input_dim, concat=u_test_t[:test_len]*renorm_factor, renorm_factor=renorm_factor)

    new_pred_horizon = pred_horizon_range * (n_rec+1)
    out_test = torch.zeros(N_test, input_dim * new_pred_horizon).to(device)
    for pred_horizon in range(1, new_pred_horizon+1):
        out_test[:, (pred_horizon-1)*input_dim:pred_horizon*input_dim] = torch.roll(u_test_t, -pred_horizon, dims=0)
    out_test = out_test[:test_len, :]
    truth = out_test[:test_len, :].reshape(test_len, (n_rec+1)*pred_horizon_range, input_dim)

    rec_pred = rec_pred_data.reshape(test_len, (n_rec+1)*pred_horizon_range, input_dim)

    diff = rec_pred - truth.cpu()
    rc_mse[:, :, seed] = torch.mean(diff**2, dim=2)

100%|██████████| 10/10 [02:30<00:00, 15.01s/it]


In [3]:
np.save('out/' + today + '_ks_rc_mse_nres' + str(n_res), rc_mse)

# Structured Reservoir Computing

In [4]:
N_seed = 10

L = 22 / (2 * np.pi)  # length
N = 100  # space discretization step
dt = 0.25  # time discretization step
N_train = 70000
N_test = 5000
N_init = 1000  # remove the initial points
tend = (N_train + N_test) * dt + N_init

n_rec = 600
test_len = 2000
src_mse = torch.zeros(test_len, n_rec+1, N_seed)

for seed in tqdm(range(N_seed)):
    np.random.seed(seed)
    dns = kuramoto.KS(L=L, N=N, dt=dt, tend=tend)
    dns.simulate()
    u = dns.uu[N_init:] / np.sqrt(N)
    [u_train, u_test, _] = np.split(u, [N_train, N_train+N_test], axis=0)
    u_train_t = torch.from_numpy(u_train).to(device)
    u_test_t = torch.from_numpy(u_test).to(device)
    input_len, input_dim = u_train_t.shape
    
    # RC parameters
    n_res = 1024-input_dim
    input_scale = 0.4
    res_scale = 0.9
    bias_scale = 0.4
    renorm_factor = 1.1 * np.sqrt(input_dim / n_res)
    alpha = 1e-2
    leak_rate = 1

    # Generation of train data
    pred_horizon_range = 1
    out_train = torch.zeros(input_len, input_dim * pred_horizon_range).to(device)
    for pred_horizon in range(1, pred_horizon_range+1):
        out_train[:, (pred_horizon-1)*input_dim:pred_horizon*input_dim] = torch.roll(u_train_t, -pred_horizon, dims=0)

#     print("Training forward")
    torch.manual_seed(seed)
    bias = bias_scale * torch.randn(n_res).to(device)
    model = reservoir.ESN(input_dim, res_size=n_res, res_scale=res_scale, input_scale=input_scale, 
                          f='erf', leak_rate=leak_rate, bias=bias, random_projection='structured', seed=seed)
    X = model.forward(u_train_t).to(device)
    X = torch.cat((X, u_train_t*renorm_factor), dim=1)
#     print("Training weights")
    output_w = model.train(X, out_train, alpha=alpha)
    
#     print("Testing forward")
    test_len = 2000
    Xtest = model.forward(u_test_t[:test_len]).to(device)

#     print("Testing recursive prediction")
    n_rec = 600
    rec_pred_data = model.rec_pred(Xtest, output_w, n_rec, input_dim, concat=u_test_t[:test_len]*renorm_factor, renorm_factor=renorm_factor)

    new_pred_horizon = pred_horizon_range * (n_rec+1)
    out_test = torch.zeros(N_test, input_dim * new_pred_horizon).to(device)
    for pred_horizon in range(1, new_pred_horizon+1):
        out_test[:, (pred_horizon-1)*input_dim:pred_horizon*input_dim] = torch.roll(u_test_t, -pred_horizon, dims=0)
    out_test = out_test[:test_len, :]
    truth = out_test[:test_len, :].reshape(test_len, (n_rec+1)*pred_horizon_range, input_dim)

    rec_pred = rec_pred_data.reshape(test_len, (n_rec+1)*pred_horizon_range, input_dim)

    diff = rec_pred - truth.cpu()
    src_mse[:, :, seed] = torch.mean(diff**2, dim=2)

100%|██████████| 10/10 [03:12<00:00, 19.29s/it]


In [5]:
np.save('out/' + today + '_ks_src_mse_nres' + str(n_res), src_mse)

## Recurrent Kernels

In [6]:
N_seed = 10

L = 22 / (2 * np.pi)  # length
N = 100  # space discretization step
dt = 0.25  # time discretization step
N_train = 70000
N_test = 5000
N_init = 1000  # remove the initial points
tend = (N_train + N_test) * dt + N_init

n_rec = 600
test_len = 2000
rk_mse = torch.zeros(test_len, n_rec+1, N_seed)

for seed in tqdm(range(N_seed)):
    np.random.seed(seed)
    dns = kuramoto.KS(L=L, N=N, dt=dt, tend=tend)
    dns.simulate()
    u = dns.uu[N_init:] / np.sqrt(N)
    [u_train, u_test, _] = np.split(u, [N_train, N_train+N_test], axis=0)
    u_train_t = torch.from_numpy(u_train).to(device)
    u_test_t = torch.from_numpy(u_test).to(device)
    input_len, input_dim = u_train_t.shape
    
    # RK parameters
    input_scale = 0.4
    res_scale = 0.9
    bias_scale = 0.4
    renorm_factor = 1.1
    alpha = 3e-3
    n_iter = 50

#     print("Training forward")
    model = reckernel.RecKernel(function='arcsin', res_scale=res_scale, input_scale=input_scale, n_iter=n_iter,
                               bias_scale=bias_scale, memory_efficient=False)

    step = 10
    in_len = (input_len - n_iter) // step
    concat_train = torch.zeros(in_len, n_iter, input_dim).to(device)
    for i in range(n_iter):
        concat_train[:, i, :] = u_train_t[i:i+step*in_len:step, :]
    K = model.forward(concat_train).to(device)

    final_train = concat_train[:, -1, :].reshape(in_len, input_dim)
    final_input_gram = final_train @ final_train.T
    K += renorm_factor * final_input_gram

#     print("Training weights")
    pred_horizon_range = 1
    out_train = torch.zeros(in_len, input_dim * pred_horizon_range).to(device)
    for pred_horizon in range(1, pred_horizon_range+1):
        out_train[:, (pred_horizon-1)*input_dim:pred_horizon*input_dim] = torch.roll(
            u_train_t, -pred_horizon, dims=0)[n_iter-1:n_iter-1+step*in_len:step, :]

    output_w = model.train(K, out_train, alpha=alpha)

#     print("Testing forward")
    test_len = 2000
    concat_test = torch.zeros(test_len, n_iter, input_dim).to(device)
    for i in range(n_iter):
        concat_test[:, i, :] = u_test_t[i:i+test_len, :]
    Ktest, diag_res_train, diag_res_test = model.forward_test(concat_train, concat_test, bypass=True)

#     print("Testing recursive prediction")
    n_rec = 600
    rec_pred_data = model.rec_pred_concat(Ktest, concat_train, concat_test, output_w, n_rec,
                                          diag_res_train, diag_res_test, renorm_factor)
    
    new_pred_horizon = pred_horizon_range * (n_rec+1)
    out_test = torch.zeros(test_len, input_dim * new_pred_horizon).to(device)
    for pred_horizon in range(1, new_pred_horizon+1):
        out_test[:, (pred_horizon-1)*input_dim:pred_horizon*input_dim] = torch.roll(u_test_t[n_iter:n_iter+test_len, :], -pred_horizon, dims=0)
    out_test = out_test[:test_len, :]
    truth = out_test[:test_len, :].reshape(test_len, (n_rec+1)*pred_horizon_range, input_dim)

    rec_pred = rec_pred_data.reshape(test_len, (n_rec+1)*pred_horizon_range, input_dim)

    diff = rec_pred - truth.cpu()
    rk_mse[:, :, seed] = torch.mean(diff**2, dim=2)

100%|██████████| 10/10 [1:13:23<00:00, 440.32s/it]


In [7]:
np.save('out/' + today + '_ks_src_mse_nres' + str(n_res), rk_mse)